# Práctica Integrada: Pipeline de Preprocesamiento para Análisis de Feedback en E-Commerce

Esta práctica busca crear un pipeline robusto que preserve formatos útiles y limpie lo irrelevante.

## Fase 0: Importacion de librerias

In [1]:
%pip install emoji
%pip install dateparser

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import json
import re
import emoji
import dateparser
import spacy

## Fase 1: Limpieza Contextual de Texto

In [3]:
# Cargar datos
with open("validation_samples.json", "r", encoding="utf-8") as f:
    samples = json.load(f)
print(samples[0])


{'input': '🔥¡OFERTA! Compre 2x zapatos Nike a $99.99 (antes $150) 👟. ¡Válido hasta el 30/11/2023! Visita https://marketmind.com/oferta-nike. Atención @MariaP: ¿Envío gratis? 😃 #ModaDeportiva2023.'}


Texto de muestra a limpiar

`🔥¡OFERTA! Compre 2x zapatos Nike a $99.99 (antes $150) 👟. ¡Válido hasta el 30/11/2023! Visita https://marketmind.com/oferta-nike. Atención @MariaP: ¿Envío gratis? 😃 #ModaDeportiva2023.`

In [4]:

def limpiar_texto(texto):
    protegidos = []

    def proteger(match):
        protegidos.append(match.group(0))
        return f"__PROT{len(protegidos)-1}__"

    # Regex para detectar $99.99, 25%, 3/4, etc.
    texto = re.sub(r"\$?\d+(?:[.,]\d+)?%?|\d+/\d+", proteger, texto)

    # Eliminar URLs y handles
    texto = re.sub(r"https?://\S+|www\.\S+|@\w+", "", texto)
    
    # Eliminar signos de puntuación no deseados (excepto ! ? % $ /)
    texto = re.sub(r"[\.,;:()\[\]\"'\\]", "", texto)
    
    # Unificar espacios
    texto = re.sub(r"\s+", " ", texto).strip()

    for i, val in enumerate(protegidos):
        texto = texto.replace(f"__PROT{i}__", val)
    
    return texto


In [5]:

print(limpiar_texto(samples[6]["input"]))


¡Buen servicio! Entrega en 2 días hábiles Precio final $200 + IVA 21% Gracias MarketMind


## 🔄 Fase 2: Normalización de Números y Unidades

In [6]:

def normalizar_numeros(texto):
    
    # Normalizar fechas
    fechas = re.findall(r"\d{2,4}/\d{2,4}/\d{2,4}", texto)
    for fecha in fechas:
        try:
            fecha_iso = dateparser.parse(fecha).strftime("%Y-%m-%d")
            texto = texto.replace(fecha, fecha_iso)
        except:
            print("no se puede parsear fecha")
            continue 

    # Monedas
    texto = re.sub(r"\$\s?(\d+(\.\d{1,2})?)", r"<USD>\1", texto)
    texto = re.sub(r"(\d+(\.\d{1,2})?)\$", r"<USD>\1", texto)
    texto = re.sub(r"(\d+(?:[.,]\d+)?)\s?USD", r"<USD>\1", texto)
    texto = re.sub(r"(\d+(?:[.,]\d+)?)\s?EUR", r"<EUR>\1", texto)
    texto = re.sub(r"(\d+(\.\d{1,2})?)€", r"<EUR>\1", texto)
    texto = re.sub(r"€\s?(\d+(\.\d{1,2})?)", r"<EUR>\1", texto)
    
    # Unidades
    texto = re.sub(r"\b(\d+)x\b", r"\1_unidades", texto)  # 2x → 2_unidades
    texto = re.sub(r"\b(\d+)(kg|ml)\b", r"\1_\2", texto)  # 3kg → 3_kg

    # Proteger porcentajes
    porcentajes = re.findall(r"\d{1,3}%", texto)
    for i, p in enumerate(porcentajes):
        texto = texto.replace(p, f"__PORC{i}__")
    
    # Reemplazar números sueltos (≥2 dígitos) que no estén ya como unidades, monedas o fechas
    # Protegemos lo que ya se ha normalizado
    protegidos = re.findall(r"<USD>\d+(?:[\.,]\d+)?|<EUR>\d+(?:[\.,]\d+)?|\d+_(?:unidades|kg|ml)|\d{4}-\d{2}-\d{2}|__PORC\d+__", texto)
    tokens_protegidos = {p: f"__PROT{idx}__" for idx, p in enumerate(protegidos)}
    for original, marcador in tokens_protegidos.items():
        texto = texto.replace(original, marcador)

    # Ahora sí: reemplazar números sueltos ≥1 dígitos
    texto = re.sub(r"\b\d{1,}\b", "<NUM>", texto)
    # Restaurar protegidos
    for original, marcador in tokens_protegidos.items():
        texto = texto.replace(marcador, original)

    # Restaurar porcentajes
    for i, p in enumerate(porcentajes):
        texto = texto.replace(f"__PORC{i}__", p)

    return texto


In [7]:
texto = limpiar_texto(samples[6]["input"])
print(normalizar_numeros(texto))


¡Buen servicio! Entrega en <NUM> días hábiles Precio final <USD>200 + IVA 21% Gracias MarketMind


## 🔤 Fase 3: Normalización de Mayúsculas con Reconocimiento de Entidades

In [8]:
!python -m spacy download es_core_news_lg

     -------------------------------------- 568.0/568.0 MB 6.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
def proteger_etiquetas(texto):
    etiquetas = re.findall(r"<(?:USD|EUR)>\d+(?:[.,]\d+)?", texto)
    numericos = re.findall(r"<NUM>?", texto)
    hashtags = re.findall(r"#\w+", texto)

    protegidos = etiquetas + numericos + hashtags
    tokens_protegidos = {etiqueta: f"PROT{idx}XYZ" for idx, etiqueta in enumerate(protegidos)}
    
    for original, marcador in tokens_protegidos.items():
        texto = texto.replace(original, marcador)

    return texto, tokens_protegidos

def restaurar_etiquetas(texto, tokens_protegidos):
    for original, marcador in tokens_protegidos.items():
        texto = texto.replace(marcador, original)
    return texto


In [10]:
nlp = spacy.load("es_core_news_lg")

marcas_manual = {"Nike", "Zara", "iPhone", "Canon"}

def normalizar_mayusculas(texto):
    texto, tokens_protegidos = proteger_etiquetas(texto)
    doc = nlp(texto)
    resultado = []
    
    for token in doc:
        # print(token.text, token.pos_, token.text.isalpha(), token.text.istitle(), token.ent_type_)
        if token.text in tokens_protegidos.values():
            resultado.append(token.text)
        elif token.text in marcas_manual:
            resultado.append(token.text)
        elif token.text.isalpha() and token.text.istitle():
            resultado.append(token.text.lower())
        elif token.ent_type_ in {"ORG", "LOC", "PRODUCT", "GPE"}:
            resultado.append(token.text)
        elif token.text.startswith("#"):
            resultado.append(token.text)
        else:
            resultado.append(token.text.lower()) 
    
    
    texto_final = " ".join(resultado)
    texto_final = restaurar_etiquetas(texto_final, tokens_protegidos)
    return texto_final


In [11]:

texto_final = normalizar_mayusculas(normalizar_numeros(limpiar_texto(samples[6]["input"])))
print(texto_final)


¡ buen servicio ! entrega en <NUM> días hábiles precio final <USD>200 + iva 21% gracias MarketMind


In [12]:
output = []

for texto in samples:
    extraido = texto["input"]
    limpiado = normalizar_mayusculas(normalizar_numeros(limpiar_texto(extraido)))
    output.append({"output": limpiado})
    
with open("output_limpiado.json", "w", encoding="utf-8") as outfile:
    json.dump(output, outfile, ensure_ascii=False, indent=4)
